In [1]:
import pandas as pd 

In [96]:
chunksize = 10 ** 3
df_full = pd.read_csv("AllArticles.csv", chunksize=chunksize, header=None)
journal_df = pd.read_csv("journalslist.csv")

In [97]:
i = 0
for chunk in df_full:
    df_chunked = chunk.iloc[:, [1, 2, 9, 10, 13]]
    df_chunked.columns = ["title", "author", "url", "abstract", "journal_id"]
    i+=1
    if i == 10:
        break


In [98]:
# journal_df[journal_df["journal_id"]==1]["journal"].values[0]
df_chunked["field"] = df_chunked["journal_id"].apply(lambda x: journal_df[journal_df["journal_id"]==x]["field"].values[0])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [99]:
df_chunked

,title,author,url,abstract,journal_id,field
9000,Credit Rationing?,Dan Bernhardt,https://www.aeaweb.org/articles?id=10.1257/aer...,NaN,1,Economics
9001,An algebraic theory of portfolio allocation,David A. Hennessy;Harvey E. Lapan,https://link.springer.com/article/10.1007/s001...,"Using group and majorization theory, we explor...",67,Economics
9002,General Equilibrium Impacts of a Federal Clean...,Lawrence H. Goulder;Marc Hafstead;Roberton C. ...,http://rff.org/research/publications/general-e...,Economists have tended to view emissions prici...,82,Economics
9003,An algorithm for envy-free allocations in an e...,Flip Klijn,https://link.springer.com/article/10.1007/s003...,This paper studies envy-free allocations for e...,61,Economics
9004,Getting Cars Off the Road: The Cost-Effectiven...,Maureen L. Cropper;Yi Jiang;Anna Alberini;Patr...,http://rff.org/research/publications/getting-c...,Ground-level ozone remains a serious problem i...,82,Economics
9005,"A nonlinear long memory model, with an applica...",Dick Van Dijk;Philip Hans Franses;Richard Paap,http://www.sciencedirect.com/science/article/p...,Two important empirical features of US unemplo...,17,Economics;Applied Mathematics
9006,Futures-Based Measures Of Monetary Policy And ...,Inekwe John Nkwoma,https://www.cambridge.org/core/product/56B4B5E...,We estimate the effects of anticipated and una...,72,Economics
9007,Credit Relationships and Business Bankruptcy d...,Mary Eschelbach Hansen;Nicolas L. Ziebarth,https://www.aeaweb.org/articles?id=10.1257/mac...,Credit relationships are sticky. Stickiness ma...,5,Economics
9008,"A Note on John Kain's “Housing Segregation, Ne...",Paul Offner;Daniel H. Saks,https://academic.oup.com/qje/article/85/1/147/...,"I. Theory, 147.—II. Empirical results, 149.—II...",44,Economics
9009,An allocation rule for dynamic random network ...,Jean-François Caulier;Michel Grabisch;Agnieszk...,https://link.springer.com/article/10.1007/s001...,Most allocation rules for network games presen...,67,Economics


In [133]:
import gensim
import spacy
# https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

In [106]:
spacy.load('en')
# python -m spacy download en
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [107]:
import nltk
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/arkin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [108]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    return tokens

In [124]:
def apply_tokenization(r):
    title = r["title"]
    abstract = r["abstract"]
    word = ""
    if not isinstance(title, float):
        word += title 
    if not isinstance(abstract, float):
        word += abstract
    return prepare_text_for_lda(word)

In [125]:
text_data = df_chunked.apply(lambda x: apply_tokenization(x), axis=1)

In [128]:
# Prolly optimize this 
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [131]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.007*"model" + 0.006*"results" + 0.006*"effects" + 0.006*"policy"')
(1, '0.008*"paper" + 0.007*"countries" + 0.006*"changes" + 0.006*"model"')
(2, '0.008*"information" + 0.005*"birth" + 0.005*"results" + 0.005*"price"')
(3, '0.008*"education" + 0.008*"market" + 0.006*"labor" + 0.005*"demand"')
(4, '0.007*"development" + 0.007*"democracy" + 0.007*"model" + 0.006*"health"')
(5, '0.008*"policy" + 0.006*"model" + 0.005*"unemployment" + 0.005*"market"')
(6, '0.007*"model" + 0.006*"public" + 0.005*"workers" + 0.005*"market"')
(7, '0.011*"income" + 0.007*"paper" + 0.007*"inequality" + 0.007*"model"')
(8, '0.006*"paper" + 0.005*"based" + 0.005*"model" + 0.004*"distribution"')
(9, '0.010*"model" + 0.008*"effects" + 0.006*"paper" + 0.006*"effect"')


In [132]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
